In [1]:
import os
import tqdm
import h5py
import xarray as xr
from dask.distributed import Client
from DASStore.modules import convert

# Taking a function from 
https://github.com/UW-ESS-DS/MLGeo2022_velgueta/blob/main/dasquakes.py

In [2]:
import numpy as np
import pandas as pd
import datetime
import h5py
import glob
from scipy.signal import detrend
from numpy.fft import fftshift, fft2, fftfreq
from datetime import datetime as DT
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [64]:
def sintela_to_datetime(sintela_times):
    '''
    returns an array of datetime.datetime 
    ''' 
    days1970 = datetime.date(1970, 1, 1).toordinal()

    # Vectorize everything
    converttime = np.vectorize(datetime.datetime.fromordinal)
    addday_lambda = lambda x : datetime.timedelta(days=x)
    adddays = np.vectorize(addday_lambda )
    
    day = days1970 + sintela_times/1e6/60/60/24
    thisDateTime = converttime(np.floor(day).astype(int))
    dayFraction = day-np.floor(day)
    thisDateTime = thisDateTime + adddays(dayFraction)

    return thisDateTime

In [65]:
def get_file_number(pth,prefix,t0,verbose=False):
    
    datestr = '{d.year}-{d.month:02}-{d.day:02}_{d.hour:02}-{d.minute:02}'.format(d=t0)

    file = f"{pth}{prefix}_{datestr}*.h5"
    if verbose:
        print(file)

    if len(glob.glob(file)) > 0:
        file_list = glob.glob(file)[0]
#         print(glob.glob(file))
        file_number = file_list.split('_')[-1]
        file_number = file_number.split('.')[0]
        file_number = int(file_number)
        return file_number
    else:
        return -1

In [66]:
def open_sintela_file(file_base_name,t0,pth,
                      chan_min=0,
                      chan_max=-1,
                      number_of_files=1,
                      verbose=False,
                      pad=False):

    data = np.array([])
    time = np.array([])

    
    dt = datetime.timedelta(minutes=1) # Assume one minute file duration
    this_files_date = t0
    
    for i in range(number_of_files):

        file_number = get_file_number(pth,file_base_name,this_files_date,verbose=verbose)
        print('file_number is ', file_number)
        if file_number == -1:
            raise ValueError('Failed to find file number.')
#             return [-1], [-1], [-1]
        date_str = this_files_date.strftime("%Y-%m-%d_%H-%M") + "-00"
        this_file = f'{pth}{file_base_name}_{date_str}_UTC_{file_number:06}.h5'
        print('this_file is', this_file)
        
        try:
            f = h5py.File(this_file,'r')
            this_data = np.array(
                f['Acquisition/Raw[0]/RawData'][:,chan_min:chan_max])
            this_time = np.array(
                f['Acquisition/Raw[0]/RawDataTime'])
            
            if i == 0:
                time = sintela_to_datetime(this_time)
                data = this_data
                attrs=dict(f['Acquisition'].attrs)
            else:
                data = np.concatenate((data, this_data ))
                time = np.concatenate((time, this_time ))
                
        except Exception as e: 
            print('File problem with: %s'%this_file)
            print(e)
            
        this_files_date = this_files_date + dt
          
    return data, time, attrs

# Creating my function

In [4]:
def get_sampling_rate(file_base_path, 
                      chan_min=0,
                      chan_max=-1, 
                      verbose=False,
                      pad=False):
    data = np.array([])
    time = np.array([])
    collecting_samplerate = []

        
    dt = datetime.timedelta(minutes=1) # Assume one minute file duration
    file_list = os.listdir(file_base_path)
    
    for i in range(len(file_list)):
        if file_list[i].endswith(".h5"):
            this_file = h5_files_path + file_list[i]
        
            try:
                f = h5py.File(this_file,'r')
                attrs = dict(f['Acquisition'].attrs)
                collecting_samplerate.append(attrs['PulseRate'])
#                 this_data = np.array(f['Acquisition/Raw[0]/RawData'][:,chan_min:chan_max])
#                 this_time = np.array(f['Acquisition/Raw[0]/RawDataTime'])
#                 if i == 0:
#                     time = sintela_to_datetime(this_time)
#                     data = this_data
#                     attrs = dict(f['Acquisition'].attrs)
#                     print('attrs right now', i, attrs)
#                 else:
#                     data = np.concatenate((data, this_data ))
#                     time = np.concatenate((time, this_time ))
#                 collecting_samplerate.append(attrs['PulseRate'])


            except Exception as e: 
                print('File problem with: %s'%this_file)
                print(e)
    
    return collecting_samplerate

# using original code by professor

In [81]:
# using original code by professor
prefix = 'whidbey'
t0 = datetime.datetime(2022, 10, 19, 2, 38, 0)
t1 = datetime.datetime(2022, 10, 20, 9, 28, 0)
record_length = 1 # minutes
data,dates,attrs = open_sintela_file(prefix,
                                     t0,
                                     h5_files_path,
                                     number_of_files=record_length,
                                     verbose=True)

/Users/aishwaryasingh/ResearchProject/samplefiles/whidbey_2022-10-19_02-38*.h5
file_number is  275
this_file is /Users/aishwaryasingh/ResearchProject/samplefiles/whidbey_2022-10-19_02-38-00_UTC_000275.h5


In [82]:
GL = attrs['GaugeLength']
fs = attrs['PulseRate']
dx = attrs['SpatialSamplingInterval']
x_max=data.shape[1] * dx
print(data.shape)
print(fs)
print(dx)
print(GL)

(30000, 1720)
500.0
6.38095235824585
6.38095235824585


In [83]:
data,dates,attrs = open_sintela_file(prefix,
                                     t1,
                                     h5_files_path,
                                     number_of_files=record_length,
                                     verbose=True)
GL = attrs['GaugeLength']
fs = attrs['PulseRate']
dx = attrs['SpatialSamplingInterval']
x_max=data.shape[1] * dx
print(data.shape)
print(fs)
print(dx)
print(GL)

/Users/aishwaryasingh/ResearchProject/samplefiles/whidbey_2022-10-20_09-28*.h5
file_number is  2124
this_file is /Users/aishwaryasingh/ResearchProject/samplefiles/whidbey_2022-10-20_09-28-00_UTC_002124.h5
(120000, 1720)
2000.0
6.38095235824585
6.38095235824585


# Creating my own code

In [5]:
# h5_files_path = '/Users/aishwaryasingh/ResearchProject/samplefiles/'
h5_files_path = '/Users/stlp/Desktop/samplefiles/'

In [6]:
os.listdir(h5_files_path)

['whidbey_2022-10-19_00-21-00_UTC_000138.h5',
 'whidbey_2022-10-19_00-15-00_UTC_000132.h5',
 'whidbey_2022-10-20_15-54-00_UTC_002510.h5',
 '.DS_Store',
 'whidbey_2022-10-20_15-52-00_UTC_002508.h5',
 'whidbey_2022-10-21_11-57-00_UTC_003711.h5',
 'DAS_file_details.csv',
 'whidbey_2022-10-21_11-56-00_UTC_003710.h5',
 'whidbey_2022-10-20_15-53-00_UTC_002509.h5',
 'example.zarr',
 'whidbey_2022-10-19_00-11-00_UTC_000128.h5',
 'whidbey_2022-10-19_00-13-00_UTC_000130.h5',
 'whidbey_2022-10-21_11-55-34_UTC_003709.h5',
 'whidbey_2022-10-21_11-54-00_UTC_003709.h5',
 'whidbey_2022-10-19_00-12-00_UTC_000129.h5',
 'whidbey_2022-10-20_15-51-00_UTC_002507.h5',
 'whidbey_2022-10-19_00-14-00_UTC_000131.h5',
 'whidbey_2022-10-20_15-50-00_UTC_002506.h5',
 'whidbey_2022-10-19_00-19-00_UTC_000136.h5',
 'whidbey_2022-10-19_00-10-52_UTC_000127.h5',
 'whidbey_2022-10-19_00-16-00_UTC_000133.h5']

In [129]:
file_list = os.listdir(h5_files_path)
for i in (range(len(file_list))):
    if file_list[i].endswith(".h5"):
        print(i)
        this_file = h5_files_path + file_list[i]
        print(this_file)

0
/Users/aishwaryasingh/ResearchProject/samplefiles/whidbey_2022-10-20_09-28-00_UTC_002124.h5
2
/Users/aishwaryasingh/ResearchProject/samplefiles/whidbey_2022-10-19_02-38-00_UTC_000275.h5


In [7]:
get_sampling_rate(file_base_path = h5_files_path)

[500.0,
 500.0,
 2000.0,
 2000.0,
 2000.0,
 2000.0,
 2000.0,
 500.0,
 500.0,
 2000.0,
 2000.0,
 500.0,
 2000.0,
 500.0,
 2000.0,
 500.0,
 500.0,
 500.0]

# Creating the pandas dataframe

Goal is to create a csv file
* of all file names
* dimensions in time and distance
* sampling rate

dont know how to do this - Start time and end time of the file

In [8]:
File_Name = []
Time_dimension = []
Dist_dimension = []

file_list = os.listdir(h5_files_path)

In [9]:
for i in range(len(file_list)):
    if file_list[i].endswith(".h5"):
        File_Name.append(file_list[i])
        file = h5py.File(h5_files_path+file_list[i], 'r')
        Time_dimension.append(file['Acquisition']['Raw[0]']['RawData'].shape[0])
        Dist_dimension.append(file['Acquisition']['Raw[0]']['RawData'].shape[1])

In [10]:
File_Name

['whidbey_2022-10-19_00-21-00_UTC_000138.h5',
 'whidbey_2022-10-19_00-15-00_UTC_000132.h5',
 'whidbey_2022-10-20_15-54-00_UTC_002510.h5',
 'whidbey_2022-10-20_15-52-00_UTC_002508.h5',
 'whidbey_2022-10-21_11-57-00_UTC_003711.h5',
 'whidbey_2022-10-21_11-56-00_UTC_003710.h5',
 'whidbey_2022-10-20_15-53-00_UTC_002509.h5',
 'whidbey_2022-10-19_00-11-00_UTC_000128.h5',
 'whidbey_2022-10-19_00-13-00_UTC_000130.h5',
 'whidbey_2022-10-21_11-55-34_UTC_003709.h5',
 'whidbey_2022-10-21_11-54-00_UTC_003709.h5',
 'whidbey_2022-10-19_00-12-00_UTC_000129.h5',
 'whidbey_2022-10-20_15-51-00_UTC_002507.h5',
 'whidbey_2022-10-19_00-14-00_UTC_000131.h5',
 'whidbey_2022-10-20_15-50-00_UTC_002506.h5',
 'whidbey_2022-10-19_00-19-00_UTC_000136.h5',
 'whidbey_2022-10-19_00-10-52_UTC_000127.h5',
 'whidbey_2022-10-19_00-16-00_UTC_000133.h5']

In [11]:
Time_dimension

[30000,
 30000,
 120000,
 120000,
 120000,
 120000,
 120000,
 30000,
 30000,
 50648,
 98856,
 30000,
 120000,
 30000,
 120000,
 30000,
 3580,
 25176]

In [12]:
Sampling_Rate = get_sampling_rate(file_base_path = h5_files_path)

In [13]:
len(File_Name), len(Time_dimension), len(Dist_dimension), len(Sampling_Rate)

(18, 18, 18, 18)

In [14]:
data = {'File_Name': File_Name,
        'Time_dimension': Time_dimension,
        'Dist_dimension': Dist_dimension,
        'Sampling_Rate': Sampling_Rate}

In [15]:
DAS_file_details = pd.DataFrame(data)

In [17]:
DAS_file_details

,File_Name,Time_dimension,Dist_dimension,Sampling_Rate
0,whidbey_2022-10-19_00-21-00_UTC_000138.h5,30000,1721,500.0
1,whidbey_2022-10-19_00-15-00_UTC_000132.h5,30000,1721,500.0
2,whidbey_2022-10-20_15-54-00_UTC_002510.h5,120000,1721,2000.0
3,whidbey_2022-10-20_15-52-00_UTC_002508.h5,120000,1721,2000.0
4,whidbey_2022-10-21_11-57-00_UTC_003711.h5,120000,1721,2000.0
5,whidbey_2022-10-21_11-56-00_UTC_003710.h5,120000,1721,2000.0
6,whidbey_2022-10-20_15-53-00_UTC_002509.h5,120000,1721,2000.0
7,whidbey_2022-10-19_00-11-00_UTC_000128.h5,30000,1721,500.0
8,whidbey_2022-10-19_00-13-00_UTC_000130.h5,30000,1721,500.0
9,whidbey_2022-10-21_11-55-34_UTC_003709.h5,50648,1721,2000.0


In [ ]:
# DAS_file_details.to_csv('DAS_file_details.csv', index = False)